## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
tres = 'monthly'
a1,a2,b1,b2 = [240,380,125,320]
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.TEMP.024901-031812.nc'
dset = xr.open_dataset(filename)
dz = dset['dz'].isel(z_t=range(0, 35))/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1, a2), nlon=range(b1, b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
dset.close()

In [4]:
dvol = dz*darea
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")

In [5]:
for varr in ['O2','AOU','T','S']:
    print(varr)
    filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_1948-2017_{tres}.nc'
    ds = xr.open_dataset(filename)
    dsf = xr.Dataset()
    for ii in [0,6,12]:
        print(ii)
        Phi_i = ds[f'Phi_{varr}_{ii}']*dvol
        # select a vertical layer
        dsf[f'Phi_{varr}_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
        dsf[f'Phi_{varr}_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))
        #
    dsf.load()
    dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_vertical_mean_1948-2017_{tres}.nc', mode='w')

O2
0
6
12
AOU
0
6
12
T
0
6
12
S
0
6
12


## calculate LME-averaged Phi 

In [6]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask'].isel(nlat=range(a1, a2), nlon=range(b1, b2))

In [7]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]
nmon = dsf.dims['time']
#nmon = dsf.dims['year']
nmon

840

In [9]:
for varr in ['O2','AOU','T','S']:
    print(varr)
    filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_vertical_mean_1948-2017_{tres}.nc'
    dsf = xr.open_dataset(filename)
    #
    boxPhi = xr.Dataset()
    #for ii in range(13):
    for ii in [0,6,12]:
        print(ii)
        phi_box1 = np.ma.zeros([len(lme), nmon])
        phi_box1.mask = True
        phi_box2 = phi_box1.copy()
        for box,hh in zip(lme, range(len(lme))):
            latloc, lonloc = np.where(lme_mask == box)
            ar1 = np.ma.zeros([len(latloc),nmon]);ar1.mask=True
            ar2 = ar1.copy()
            for bb in range(len(latloc)):
                ar1[bb] = dsf[f'Phi_{varr}_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
                ar2[bb] = dsf[f'Phi_{varr}_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            phi_box1[hh] = np.nanmean(ar1,axis=0)
            phi_box2[hh] = np.nanmean(ar2,axis=0)
        #
        boxPhi[f'Phi_{varr}_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), time=('time', dsf.coords['time'].values)), dims=('lme','time'))
        boxPhi[f'Phi_{varr}_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), time=('time', dsf.coords['time'].values)), dims=('lme','time'))
    #
    boxPhi.load()
    boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_LME_1948-2017_{tres}.nc', mode='w')
#

O2
0
6
12
AOU
0
6
12
T
0
6
12
S
0
6
12
